In [1]:
%env XLA_IR_DEBUG=1
%env XLA_HLO_DEBUG=1

env: XLA_IR_DEBUG=1
env: XLA_HLO_DEBUG=1


In [2]:
import torch
import torch.nn as nn
import torch_xla

In [3]:
from torch_xla.experimental.scan import scan

In [4]:
weird_tensor = torch.tensor([0.0, 0.0], device=torch_xla.device())

def step_fn(carry, x):
  new_carry = carry + x
  weird_tensor.add_(1.0)
  y = new_carry + weird_tensor
  return new_carry, y

init = torch.tensor([0.0, 0.0], device=torch_xla.device())
xs = torch.tensor([[0.0, 0.0], [0.0, 0.0], [0.0, 0.0]], device=torch_xla.device())

# Bug: because we only trace the DAG subgraph rooted at `step_fn` outputs, mutations
# to `weird_tensor` aren't captured. TODO(yifeit): How to solve this???
scan(step_fn, init, xs)

Scan is tracing user fn. HLO:
HloModule IrToHlo.18, entry_computation_layout={(f32[2]{0}, f32[2]{0}, f32[2]{0})->(f32[2]{0}, f32[2]{0})}

ENTRY %IrToHlo.18 (p0.2: f32[2], p1.3: f32[2], p2.10: f32[2]) -> (f32[2], f32[2]) {
  %p1.3 = f32[2]{0} parameter(1), metadata={op_type="xla__device_data" op_name="xla__device_data" source_file="/workspaces/torch/pytorch/xla/torch_xla/experimental/scan.py" source_line=168}
  %p0.2 = f32[2]{0} parameter(0), metadata={op_type="xla__device_data" op_name="xla__device_data" source_file="/workspaces/torch/pytorch/xla/torch_xla/experimental/scan.py" source_line=168}
  %constant.1 = f32[] constant(1), metadata={op_type="prim__Constant" op_name="prim__Constant" source_file="/tmp/ipykernel_12491/2426694750.py" source_line=4}
  %broadcast.4 = f32[2]{0} broadcast(f32[] %constant.1), dimensions={}, metadata={op_type="aten__add" op_name="aten__add.1/aten__add" source_file="/tmp/ipykernel_12491/2426694750.py" source_line=4}
  %multiply.5 = f32[2]{0} multiply(f32[2]

(tensor([0., 0.], device='xla:0'),
 tensor([[1., 1.],
         [1., 1.],
         [1., 1.]], device='xla:0'))

In [5]:
import torch
from torch.utils._pytree import tree_map, tree_flatten, tree_iter, tree_leaves, PyTree


def loopy_scan(fn, init, xs):
  """A simple scan implemented with for loops serving as reference
  implementation."""
  carry = init
  ys = []
  xs_len = len(next(iter(tree_iter(xs))))
  for i in range(xs_len):
    carry, y = fn(carry, tree_map(lambda x: x[i], xs))
    ys.append(y)
  ys = tree_map(lambda *x: torch.stack(x), *ys)
  return carry, ys


weird_tensor = torch.tensor([0.0, 0.0], device=torch_xla.device())

def step_fn(carry, x):
  new_carry = carry + x
  weird_tensor.add_(1.0)
  y = new_carry + weird_tensor
  return new_carry, y

init = torch.tensor([0.0, 0.0], device=torch_xla.device())
xs = torch.tensor([[0.0, 0.0], [0.0, 0.0], [0.0, 0.0]], device=torch_xla.device())

loopy_scan(step_fn, init, xs)

(tensor([0., 0.], device='xla:0'),
 tensor([[1., 1.],
         [2., 2.],
         [3., 3.]], device='xla:0'))